### Working with GenBank data
There's a very useful module that makes dealing with common bioinformatic sequence and other formats relatively easy. Even if it doesn't look that simple, at least it has done a lot of work that would have been needed if you implemented this from the ground up.

This is called the **BioPython** module.

You use it via the module name **Bio** that you import into your Python session or file.

NCBI changed their requirements for accessing them via the Internet to require https instead of just http. I believe this was a change that all federal government web sites were encouraged to make.

Since the NCBI website locations are embedded in the BioPython code you need to have ***Version 1.68*** or greater. The latest version from July 2017 is ***1.70*** and it handles MAF and NEXUS formats better than earlier versions.
#### Installing the BioPython module onto your computer
You can use **pip** to install this module via the bash terminal command:

**pip install biopython**

After it is installed check its version.

In [ ]:
import Bio
print "biopython version number is:", Bio.__version__

If it is version 1.68 or greater, we should be good to go.

The first thing we are going to do is define a Python function that prints summary info based on a query.

#### Click on the cell below and Run it.

In [ ]:
#!/usr/bin/env python

# usage: esummary.py "quoted_query_string"
#
# return the summary records that correspond to the query
# number of matching records written to stderr
# first line is column names of Accn TaxID Length and Title
# rest of lines are summaries with these fields for the records
# fields separated by | since that is not a likely character in Title

import sys
from Bio import Entrez
Entrez.email = "ccg@calacademy.org"

def inJupyter(): # see if we are running in an iPython or Jupyter notebook
    try:
        get_ipython
    except:
        return False
    else:
        return True

def eSummary(qryStr, return_accns = False, max_to_show = -1):
    global rec # only so we can inspect it (eg keys) later in a Jupyter notebook
    handle = Entrez.esearch(db="nucleotide", term=qryStr, retmax=10000)
    record = Entrez.read(handle)
    numIDs = int(record["Count"])
    sys.stderr.write(str(numIDs) + " results for \""+qryStr+"\"\n")
    if numIDs < 1:
        sys.exit(0)

    if inJupyter() and max_to_show==-1: max_to_show = max_jupyter_recs

    if max_to_show > 0 and numIDs > max_to_show: # limit to max_to_show
        numIDs = max_to_show

    id_list = record["IdList"]
    id_list.sort()

    found_accns = [] # list containing the found accession numbers
    if not return_accns: print "Accn | TaxonID | Length | Title"
    startIx = 0; sliceSize = min(numIDs,200) # can't send too large a request at a time, so cut it up into this size slices
    while numIDs > 0:
        ids = ",".join( id_list[startIx : startIx+sliceSize] )

        handle = Entrez.esummary(db="nucleotide", id=ids)
        records = Entrez.read(handle)

        for rec in records:
            Accn = rec['Caption']
            if not return_accns:
                print Accn, '|', rec['TaxId'], '|', rec['Length'], '|', rec['Title']
            else:
                found_accns.append(Accn)

        numIDs  -= sliceSize
        startIx += sliceSize

    if return_accns:
        return found_accns

def usage():
    sys.stderr.write('''
Usage: esummary.py \"query_str\"
    Ex: esummary.py \"Strix[Title] AND mitochondrial[Title]\"
    
''')
    sys.exit(0)

# if we are in Jupyter notebook, run an example query and limit output to 25 records
if inJupyter():
    qryStr = "Strix[Title] AND mitochondrion[Title]"
    #qryStr = "1246431282 1246431296 523582230"
    #qryStr = "KC953095 MF431746 MF431745 KU365899 MF001440"
    max_jupyter_recs = 25 # only show up to 25 results if using in Jupyter
elif len(sys.argv) < 2:
    usage()
else:
    qryStr = sys.argv[1] # ex: "Strix[Title] AND mitochondrion[Title]"

eSummary(qryStr) # get our summaries with the defined query string


When you run the Python code in the cell above, it shows 3 summary records as a test to make sure everything is connected up and working.

But the nice thing is that now that we have **def**ined the **eSummary** function, it is available to call in this notebook. There are two example queries in the following cells. Click, or use the arrow keys, to select a cell and then Run them. After that, if you'd like, add another cell below them and try a search query of your own.

In [ ]:
eSummary("KC953095 KX773380 MF431746")

In [ ]:
eSummary("Centropyge flavissimi and rRNA")

You might notice that we didn't get all 191 results above. That's because the function only shows 25 when we are in the Jupyter notebook. However, the **esummary.py** file that is in this directory will show every result when you run it from the command line.

---
I'm sure you want to know what else is considered *summary* information for a GenBank record. You can, of course, look that up in the documentation on the web. But, for the database on which we just ran our query (*nucleotide* in this case), we can look at all the information returned for the last result. We used the **rec** variable to hold each record and so just like any other Python variable we can look to see what it contains.

Run the cell below that has **print rec** in it.

In [ ]:
print rec

There's nothing special about that **rec** name, we could have called it by whatever valid Python variable name we wanted.

What you see above is how Python shows you a **dict**ionary variable. The dictionary is enclosed in set braces {} and each item in the dictionary has the format **key: value** separated by commas. The first item is named 'Status' and has the value 'live' and the last item is named 'Gi' and has the value 239583884. That is, if you ran the search above last. If you ran other searches, the keys should still be there but the values would be different.

This shows us we could add some additional fields to Accn, TaxonID, Length, Title. For example, show the CreateDate and the UpdateDate. But not a lot more than that. If we want more we need a different call to GenBank.

---
Now let's make a script that will return more of the information than esummary. It will show us an example when run. However, importantly, it will define the **efetch** function that we can use for more purposes in this notebook page.

In [ ]:
#!/usr/bin/env python

# usage: efetch.py IDs [ [-gb] | [-f <feature_name>|all [-q <qualifier_name>|all]] [-s max_seqlen] ]
#
# return the summary records that correspond to the Accession IDs 
# number of matching records written to stderr
# first line is column names of gID TaxID Length and Title
# rest of lines are summaries with these fields for the records
# fields separated by | since that is not a likely character in Title

import sys
from Bio import Entrez
Entrez.email = "ccg@calacademy.org"

from IPython.core.debugger import set_trace

def inJupyter(): # see if we are running in an iPython or Jupyter notebook
    try:
        get_ipython
    except:
        return False
    else:
        return True

def get_feature_qualifier_list(rec, feature_name, qualifier_name):
    result = []
    if not 'GBSeq_feature-table' in rec or feature_name == "":
        return result
    
    # build list of all qualifier values for feature_name[qualifier_name]
    # special case of feature_name of "all" for ALL feature_names
    # also special case for empty qualifier_name puts name of feature and name of qualifier in the list
    #   and if qualifier_name is "all" then also include value with feature and qualifier name
    special_case_qual = (qualifier_name == "" or qualifier_name.lower() == "all")
    
    for feat in rec['GBSeq_feature-table']:
        if (feature_name.lower() == "all" or feat['GBFeature_key'] == feature_name) and 'GBFeature_quals' in feat:
            for qual in feat['GBFeature_quals']:
                if qual['GBQualifier_name'] == qualifier_name:
                    result.append( qual['GBQualifier_value'] )
                elif special_case_qual: # special case: put feature name and qualifier name and maybe val in list
                    inf = feat['GBFeature_key'] + " \t" + qual['GBQualifier_name']
                    if qualifier_name.lower() == "all":
                        inf += " \t" + qual['GBQualifier_value']
                    result.append(inf)
    return result
    
def efetch(IDs, mode="xml", seq_end=-1, feat_name="", qual_name=""): #IDs is a string with comma separated values, each an ID
    global rec # only so we can inspect value (eg keys) later in a Jupyter notebook
    
    showing_feature = (feat_name != "")
    if mode == "xml" and feat_name == "":
        print "Accn | Length | UpdateDate | Title | Taxonomy"

    # if use a short seq_stop this speeds up retrieval quite a bit. but you won't get actual length values
    if seq_end <= 0:
        handle = Entrez.efetch(db="nucleotide", id=IDs, rettype="gb", retmode=mode)
    else:
        handle = Entrez.efetch(db="nucleotide", id=IDs, rettype="gb", retmode=mode, strand=1,seq_start=1,seq_stop=seq_end)

    if mode == "text":
        print(handle.read())
    else:
        records = Entrez.read(handle)
        for rec in records:
            # set_trace() #enter ipdb debugger if we want to look at records interactively in Jupyter notebook
            if not showing_feature:
                print rec['GBSeq_primary-accession'], '|', rec['GBSeq_length'], '|', rec['GBSeq_update-date'], '|', \
                      rec['GBSeq_definition'],        '|', rec['GBSeq_taxonomy']
            else:
                features = get_feature_qualifier_list(rec, feat_name, qual_name)
                for qual in features:
                    print rec['GBSeq_primary-accession'] + " \t" + qual
                    
def usage():
    sys.stderr.write('''
    Usage: efetch.py <ID1> [<ID2> ...] [ [-gb] | [-f <feature_name>|all [-q <qualifier_name>|all]] [-s max_seqlen] ]
    
    Examples:
        efetch.py KX773372 KX773303     # show Accn Length UpdateDate Title Taxonomy for each ID
        efetch.py KX773372 -gb          # show full textual GenBank output format
        efetch.py KX773372 -f source    # show name of a specific feature and each of its qualifier names
        efetch.py KX773372 -f source -q organelle # show specific feature and a specific qualifier value
        efetch.py MF431746 -f source -q all       # show specific feature and each of its qualifier names and value
        efetch.py MF431746 -f all                 # show names of each feature and each of its qualifier names
        efetch.py KX773372 -s 200       # limit the size of the sequence retrieved. this also limits which features are retrieved.

''');
    sys.exit(0)

def get_options(arg_list, min_args=2):
    class options:
        argv           = arg_list
        id_list        = []
        mode           = "xml"
        sequence_max   = -1
        feature_name   = ""
        qualifier_name = ""
        def number_of_ids(): # this is how a class method is defined
            return len(id_list)

    num_args = len(options.argv)
    if num_args < min_args:
        usage()

    lst = options.id_list
    ixArg = 0; ixLast = num_args-1;
    while ixArg < ixLast:
        ixArg += 1
        idStr = arg_list[ixArg]
        if idStr[0] != "-":
            lst.append(idStr)
        elif idStr == "-gb" or idStr == "-text":
            options.mode = "text"
        else: # get options that have a value after the option flag
            optionType = idStr[1].lower()
            if optionType in "sfq" and ixArg < ixLast: # -s, -f or -q has a value after
                ixArg += 1
                optVal = arg_list[ixArg]
                if optionType == "f":
                    options.feature_name = optVal
                elif optionType == "q":
                    options.qualifier_name = optVal
                elif optionType == "s":
                    options.sequence_max = optVal
    return options

# main code entrypoint
if inJupyter():
    efetch("KC953095,KX773380,MF431746")
else:
    ops = get_options(sys.argv)
    ids = ",".join( ops.id_list )
    
    # call efetch with comma delimited set of IDs and optionally seq max, feature and qualifier name
    efetch(ids, ops.mode, seq_end=ops.sequence_max, feat_name=ops.feature_name, qual_name=ops.qualifier_name)

In [ ]:
efetch("KC953095,KX773380,MF431746", feat_name="source", qual_name="country")

In [ ]:
efetch("KC953095,KX773380,MF431746", feat_name="source", qual_name="all")

In [ ]:
efetch("KC953095", feat_name="tRNA", qual_name="product")

In [ ]:
efetch("KC953095", feat_name="tRNA", qual_name="all")

In [ ]:
feature_name = 'source'; qualifier_name = 'country'
#feature_name = 'rRNA'; qualifier_name = 'product'
print get_feature_qualifier_list(rec, feature_name, qualifier_name)

In [ ]:
efetch("KC953095")

In [ ]:
efetch("KX773372", "text")

In [ ]:
efetch("KC953095", "text", 180)

In [ ]:
rec.keys()

In [ ]:
efetch("KX773372", feat_name="source", qual_name="all")

### Some testing code that you can skip if reading this on your own.

In [ ]:
def print_object_attrs(obj):
    attrs = [a for a in dir(obj) if not a.startswith('__') and not callable(getattr(obj,a))]
    for attr in attrs:
        print attr + ":", getattr(obj, attr)

def print_object_methods(obj):
    methods = [a for a in dir(obj) if callable(getattr(obj,a)) and not a.startswith('__')]
    for method in methods:
        print method

ops = get_options(["pgm", "KC953095","-f", "source", "-q", "all"])
ids = ",".join( ops.id_list )
#print "ops object attributes:"; print_object_attrs(ops); print #debug stmt, comment out this line to remove debug

# call efetch with comma delimited set of IDs and optionally seq max, feature and qualifier name
efetch(ids, ops.mode, seq_end=ops.sequence_max, feat_name=ops.feature_name, qual_name=ops.qualifier_name)

### Using eSummary and efetch together
The **efetch** function isn't quite as flexible as the **eSummary** function. While efetch requires a string of Accession IDs, eSummary can take any type of GenBank query.

We can get eSummary to return a list of the Accession IDs that it found and turn those into a string for efetch.

First, let's see eSummary return those IDs.

In [ ]:
accn_list = eSummary("Strix[Title] AND mitochondrion[Title]", True)
print "Here's my list:",accn_list

If you look at the first line above at the end of the statement, you'll see we added a **True** as the second parameter to the eSummary function. That tells the function to return a list of the Accession IDs instead of printing out our summary.

We capture that ID list in the variable called **accn_list** and print it.

However, we can just turn it into a comma separated string and store it in the variable accn_str

In [ ]:
accn_str = ",".join(accn_list)
print accn_str

Then we can pass accn_str to efetch and see what *source* Features they have.

In [ ]:
efetch(accn_str, feat_name="source", qual_name="all")

### There's much more you can do with the BioPython module. Check it out at *biopython.org* or search biopython in your browser.

### We'll look at some more bioinformatic examples below
Let's make a Codon to Amino Acid map using a Python dictionary data type.
Then use it to look up a few codons.

In [ ]:
def get_AA_map():
    return {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',                 
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'*', 'TAG':'*',
        'TGC':'C', 'TGT':'C', 'TGA':'*', 'TGG':'W',
    }

nt_AA_map = get_AA_map()
codon = "CGG"
print "codon {} is Amino Acid {}.".format( codon.upper(), nt_AA_map[ codon.upper() ] )

codon = "tcg"
print "codon {} is Amino Acid {}.".format( codon.upper(), nt_AA_map[ codon.upper() ] )

Now let's see if we can translate this Turkey virus gene (in accession X63408)

In [ ]:
gene = "atgccagtggtaataccctgcagaagggtgacagcaataattaagtgcaatgcacttggattgtgtatgg" + \
       "ttcgtaagatatatgattatagtattgcgagctggagtgatttaatagaggaagtagccaatatggtcct" + \
       "aatagatcacatcaataggaaacaatgtgtggagtgtcgaaaagattttgagtttatagctatatatacctcatataattag"
gene = gene.upper()

codon_nts = len(gene) - (len(gene) % 3) # only want multiple of 3 characters
nt_AA_map = get_AA_map()

translation = ""
for ix in range(0, codon_nts, 3):
    codon = gene[ix: ix+3]
    aa = nt_AA_map[codon]
    translation += aa  # equivalently in 1 line: translation += nt_AA_map[ gene[ix: ix+3] ]

print translation

Let's download two sequences from GenBank and perform a global alignment

In [ ]:
#efetch and then look in avesmito for align code used there